In [ ]:
import pandas as pd
import numpy  as np
import networkx as nx
from math import sin, cos, pi, sqrt
import random
import time
import rtsvg
rt = rtsvg.RACETrack()

_n_paths_         = 100
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_min_circle_sep_  = 30
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def createCircleDataset(n_circles=_n_circles_, n_paths=_n_paths_, radius_min=_radius_min_, radius_max=_radius_max_, min_circle_sep=_min_circle_sep_, radius_inc_test=_radius_inc_test_):
    circle_geoms = []
    def circleOverlaps(cx, cy, r):
        for _geom_ in circle_geoms:
            dx, dy = _geom_[0] - cx, _geom_[1] - cy
            d      = sqrt(dx*dx+dy*dy)
            if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
                return True
        return False
    def findOpening():
        _max_attempts_ = 100
        attempts  = 0
        cx, cy, r = random.randint(radius_max+min_circle_sep, 600-radius_max-min_circle_sep), \
                    random.randint(radius_max+min_circle_sep, 400-radius_max-min_circle_sep), random.randint(radius_min,radius_max)
        while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
            cx, cy, r = random.randint(radius_max+min_circle_sep, 600-radius_max-min_circle_sep), \
                        random.randint(radius_max+min_circle_sep, 400-radius_max-min_circle_sep), random.randint(radius_min,radius_max)
            attempts += 1
        if attempts == _max_attempts_:
            return None
        return cx, cy, r

    # Randomize the circles
    for i in range(n_circles):
        to_unpack = findOpening()
        if to_unpack is not None:
            cx, cy, r = to_unpack
            circle_geoms.append((cx,cy,r))

    # Randomize the entry point
    c0         = random.randint(0, len(circle_geoms)-1)
    cx, cy, r  = circle_geoms[c0]
    a0         = random.random() * 2 * pi
    entry_pt   = (cx+(r+_radius_inc_test_+0.5)*cos(a0),cy+(r+_radius_inc_test_+0.5)*sin(a0),c0)
                
    # Randomize the exit points
    exit_pts = []
    for i in range(n_paths):
        c1 = random.randint(0,len(circle_geoms)-1)
        while c1 == c0:
            c1 = random.randint(0,len(circle_geoms)-1)
        cx, cy, r  = circle_geoms[c1]
        a1         = random.random() * 2 * pi
        exit_pts.append((cx+(r+radius_inc_test+0.5)*cos(a1),cy+(r+radius_inc_test+0.5)*sin(a1),c1))

    return entry_pt, exit_pts, circle_geoms

_entry_pt_,_exit_pts_,_circle_geoms_ = createCircleDataset()

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#404040" fill="#a0a0a0" fill-opacity="0.2" />'
# Render Entry Points
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="3" stroke="#00af00" fill="#00af00" />'
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="6" stroke="#000000" fill="none" />'
# Render Exit Points
for i in range(len(_exit_pts_)):
    _exit_  = _exit_pts_[i]
    _color_ = rt.co_mgr.getColor(i) 
    svg += f'<circle cx="{_exit_[0]}" cy="{_exit_[1]}" r="3" stroke="{_color_}" fill="{_color_}" />'
svg_base = svg
rt.displaySVG(svg_base + '</svg>')

In [ ]:
#
# Uses Half Way Point Between All Circles (assuming the path between the two circles does not intersect another circle)
#

#
# circularPathRouter() - route exits to a single entry around circles
# - all points need to be at least circle_radius + radius_inc_test + 1.0 from the circle centers...
#
def circularPathRouter(entry_pt,                  # (x,y,circle_i) -- where circle_i is the circle index from circle_geoms
                       exit_pts,                  # [(x,y,circle_i),(x,y,circle_i),(x,y,circle_i), ...] -- where circle_i is the circle index from circle_geoms
                       circle_geoms,              # [(cx,cy,r),(cx,cy,r), ...]
                       escape_px          = 5,    # length to push the exit points (and entry point) away from circle
                       min_circle_sep     = 30,   # minimum distance between circles
                       half_sep           = 15,   # needs to be more than the radius_inc_test ... half separation (but doesn't have to be)
                       radius_inc_test    = 4,    # for routing around circles, how much to test with
                       radius_start       = 5,    # needs to be more than the radius_inc_test ... less than the min_circle_sep
                       max_pts_per_node   = 50,   # maximum points per node for the xy quad tree
                       merge_distance_min = 5):   # minimum distance necessary to merge a path into an already exiting path
    _svg_ = ['<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />']
    for _circle_ in circle_geoms:
        _svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#b0b0b0" />')
    for _circle_ in circle_geoms:
        for _other_circle_ in circle_geoms:
            if _circle_ != _other_circle_:
                _seg_ = (_circle_[:2], _other_circle_[:2])
                straight_shot = True
                i = 0
                while straight_shot and i < len(circle_geoms):
                    to_test = circle_geoms[i]
                    if to_test != _circle_ and to_test != _other_circle_:
                        d, pt = rt.segmentIntersectsCircle(_seg_, to_test)
                        if d < (to_test[2] + radius_inc_test):
                            straight_shot = False
                    i += 1
                if straight_shot:
                    _svg_.append(f'<circle cx="{(_seg_[0][0] + _seg_[1][0])/2}" cy="{(_seg_[0][1] + _seg_[1][1])/2}" r="3" fill="#ff0000" />')

    _svg_.append('</svg>')
    return ''.join(_svg_)
rt.svgObject(circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_))

In [ ]:
#
# circularPathRouter() - route exits to a single entry around circles
# - all points need to be at least circle_radius + radius_inc_test + 1.0 from the circle centers...
#
def circularPathRouter(entry_pt,                  # (x,y,circle_i) -- where circle_i is the circle index from circle_geoms
                       exit_pts,                  # [(x,y,circle_i),(x,y,circle_i),(x,y,circle_i), ...] -- where circle_i is the circle index from circle_geoms
                       circle_geoms,              # [(cx,cy,r),(cx,cy,r), ...]
                       escape_px          = 5,    # length to push the exit points (and entry point) away from circle
                       min_circle_sep     = 30,   # minimum distance between circles
                       half_sep           = 15,   # needs to be more than the radius_inc_test ... half separation (but doesn't have to be)
                       radius_inc_test    = 4,    # for routing around circles, how much to test with
                       radius_start       = 5,    # needs to be more than the radius_inc_test ... less than the min_circle_sep
                       max_pts_per_node   = 50,   # maximum points per node for the xy quad tree
                       merge_distance_min = 5):   # minimum distance necessary to merge a path into an already exiting path
    _svg_ = ['<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />']
    pts   = []

    # Circle Rings
    for _circle_ in circle_geoms:
        _svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#b0b0b0" />')
        for a in range(0,360,30):
            cx, cy, r = _circle_
            rad       = pi * a / 180.0
            pt        = (cx + cos(rad) * (r + half_sep), cy + sin(rad) * (r + half_sep))
            pts.append(pt)
            _svg_.append(f'<circle cx="{pt[0]}" cy="{pt[1]}" r="2" fill="#ff0000" />')

    # Circle Halfway Segments
    already_done = set()
    for _circle_ in circle_geoms:
        for _other_circle_ in circle_geoms:
            if _circle_ != _other_circle_:
                _seg_ = (_circle_[:2], _other_circle_[:2])
                if _seg_ in already_done:
                    continue
                already_done.add(_seg_)
                already_done.add((_seg_[1], _seg_[0]))
                straight_shot = True
                i = 0
                while straight_shot and i < len(circle_geoms):
                    to_test = circle_geoms[i]
                    if to_test != _circle_ and to_test != _other_circle_:
                        d, pt = rt.segmentIntersectsCircle(_seg_, to_test)
                        if d < (to_test[2] + radius_inc_test):
                            straight_shot = False
                    i += 1
                if straight_shot:
                    pt = ((_seg_[0][0] + _seg_[1][0])/2, (_seg_[0][1] + _seg_[1][1])/2)
                    pts.append(pt)
                    _svg_.append(f'<circle cx="{pt[0]}" cy="{pt[1]}" r="2" fill="#0000ff" />')
                    uv, l = rt.unitVector(_seg_), rt.segmentLength(_seg_)

                    d  = (l/2 - _circle_[2])/2 + _circle_[2]
                    pt = _seg_[0][0] + uv[0]*d, _seg_[0][1] + uv[1]*d
                    _svg_.append(f'<circle cx="{pt[0]}" cy="{pt[1]}" r="3" fill="none" stroke="#00bb00" />')
                    pts.append(pt)

                    d  = (l/2 - _other_circle_[2])/2 + l/2
                    pt = _seg_[0][0] + uv[0]*d, _seg_[0][1] + uv[1]*d
                    _svg_.append(f'<circle cx="{pt[0]}" cy="{pt[1]}" r="3" fill="none" stroke="#00bb00" />')
                    pts.append(pt)

    in_emst = set()
    for emst_level in range(3):
        g = nx.Graph()
        for p0 in pts:
            for p1 in pts:
                if p0 != p1 and (p0,p1) not in in_emst and (p1,p0) not in in_emst:
                    g.add_edge(p0, p1, weight=rt.segmentLength((p0,p1)))
        emst = list(nx.minimum_spanning_edges(g))
        for _tuple_ in emst:
            p0, p1 = _tuple_[0], _tuple_[1]
            in_emst.add((p0,p1))
            _svg_.append(f'<line x1="{p0[0]}" y1="{p0[1]}" x2="{p1[0]}" y2="{p1[1]}" stroke="#000000" stroke-width="0.6" />')

    _svg_.append('</svg>')
    print(len(in_emst), len(pts))
    return ''.join(_svg_), in_emst

svg_routing, emst = circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_)

rt.svgObject(svg_routing)

In [ ]:
g = nx.Graph()
x0,y0,x1,y1 = None, None, None, None
for _segment_ in emst:
    g.add_edge(_segment_[0], _segment_[1], weight=rt.segmentLength(_segment_))
    x0 = _segment_[0][0] if x0 is None else min(x0, _segment_[0][0])
    y0 = _segment_[0][1] if y0 is None else min(y0, _segment_[0][1])
    x0 = min(x0, _segment_[1][0])
    y0 = min(y0, _segment_[1][1])
    x1 = _segment_[1][0] if x1 is None else max(x1, _segment_[1][0])
    y1 = _segment_[1][1] if y1 is None else max(y1, _segment_[1][1])
    x1 = max(x1, _segment_[0][0])
    y1 = max(y1, _segment_[0][1])
qt = rt.xyQuadTree((x0,y0,x1,y1), max_pts_per_node=5)
pts = []
for _segment_ in emst:
    pts.append(_segment_[0])
    pts.append(_segment_[1])
qt.add(pts)

In [ ]:
svg = []
g_enter = qt.closest(_entry_pt_, 1)[0][1]
for pt in _exit_pts_:
    g_exit  = qt.closest(pt, 1)[0][1]
    _path_  = nx.shortest_path(g, g_exit, g_enter)
    _path_.insert(0, pt)
    _path_.append(_entry_pt_)
    svg.append(f'<path d="{rt.svgPathCubicBSpline(_path_, beta=0.95)}" stroke="#000000" stroke-width="1" fill="none" />')
rt.displaySVG(svg_base + ''.join(svg) + '</svg>')